# Logistic Regression
We continue our study of classification by examining a popular model known as logistic regression. It belongs to the class of probabilistic models and here we model the probability that a feature vector $\phi(\mathbf{x})$ belongs to a given class.

We consider the two class problem where given an input vector, $\mathbf{x}$, we seek to assign it to one of two classes $\mathcal{C}_1$ or $\mathcal{C}_2$.

We model the probability

$\begin{equation}
p\big(\mathcal{C}_1\big|\phi(\mathbf{x})\big)=\sigma\big(\mathbf{w}^T\phi(\mathbf{x})\big)
\end{equation}
$

Where $\sigma(.)$ is the logistic sigmoid function defined as

$
\begin{equation}
\sigma(a)=\frac{1}{1+e^{-a}}
\end{equation}
$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
a = np.linspace(-10, 10, 100)
plt.plot(a, 1 / (1 + np.exp(-a)))
plt.grid(True)
plt.xlabel(r'$a$', fontsize=14)
plt.ylabel(r'$\sigma(a)$', fontsize=14)

Since the probabilities must sum to one, $p\big(\mathcal{C}_2\big|\phi(\mathbf{x})\big)=1-p\big(\mathcal{C}_1\big|\phi(\mathbf{x})\big)$

For a given data set $\{\mathbf{x}_n, t_n\}$ where $t_n\in\{0, 1\}$, we estimate the parameters by maximum likelihood.

### Maximum Likelihood Detour
For a single data point $\mathbf{x}_n$, let $y_n = \sigma\big(\mathbf{w}^T\phi(\mathbf{x}_n)\big)$. 

We can write

$
\begin{equation}
p(t_n|\mathbf{w},\mathbf{x}_n)=\left\{ \begin{array}{ll}
y_n & \textrm{ $t_n = 1$}\\
1-y_n & \textrm{$t_n=0$}
\end{array} \right.
\end{equation}
$

This can be written compactly as 

$
\begin{equation}
p(t_n|\mathbf{w},\mathbf{x}_n)=y_n^{t_n}(1-y_n)^{1-t_n}
\end{equation}
$

The likelihood function is the 

$
\begin{equation}
p(\mathbf{t}|\mathbf{w}) = \prod_{n=1}^Ny_n^{t_n}(1-y_n)^{1-t_n}
\end{equation}
$

We seek the $\mathbf{w}$ that maximizes the likelihood. This is equivalent to maximizing the logarithm of the likelihood or minimizing the negative logarithm of the likelihood.

We define the error function as the negative logarithm of the likelihood

We have

$
\begin{equation}
E(\mathbf{w}) = -\log p(\mathbf{t}|\mathbf{w})=-\sum_{n=1}^N\{t_n\ln y_n + (1-t_n)\ln(1-y_n)\}
\end{equation}
$

This is known as the cross-entropy error function.

To proceed, we compute the gradient of this error function. We get 

$
\begin{equation}
\nabla E(\mathbf{w})=\sum_{n=1}^N(y_n-t_n)\phi(\mathbf{x}_n) 
\end{equation}
$

We can proceed by stochastic gradient descent where we have

$
\begin{equation}
\mathbf{w}^{(\tau + 1)}=\mathbf{w}^{(\tau)} - \eta\nabla E_n(\mathbf{w}) 
\end{equation}
$

where

$E_n(\mathbf{w}) =(y_n-t_n)\phi(\mathbf{x}_n) $ 

### Example

Now let us try things out on our favourite Gaussian blob data set.

In [ ]:
num_class = 2
num_per_class = 20
var = 0.1
means = [[1,1],[-1,-1]]
cov = [[var, 0], [0, var]] 
X = np.array([])
y = []
class_color = ['b','r']

for class_index in range(num_class):
    class_data = np.random.multivariate_normal(means[class_index],
                                               cov, 
                                               num_per_class)
    X = np.vstack([X, class_data]) if X.size else class_data
    y = np.concatenate((y, np.ones(num_per_class) * class_index))
    plt.plot(class_data[:, 0], 
             class_data[:,1], 
             class_color[class_index] + 'o')
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel(r'$x_1$', fontsize=14)
    plt.ylabel(r'$x_2$', fontsize=14)

We will learn the weight vector by stochastic gradient descent. 

In [ ]:
def cross_entropy_gradient(input_vector, weight, target):
    '''Compute the gradient of the cross entropy error function
    Args:
        input_vector: the input vector x
        weight: the weight vector w
        target: the target value 1 or 0
    Returns:
        gradient: the gradient at the input vector
    '''
    yn = 1 / (1 + np.exp(-np.dot(weight, input_vector)))
    return (yn - target)*input_vector

To determine the decision boundary we need to have a decision rule. A reasonable rule is to classify an input vector as belonging to $\mathcal{C}_1$ if $p\big(\mathcal{C}_1\big|\phi(\mathbf{x})\big) > 0.5$. Therefore the decision boundary is given by

$\begin{equation}
\sigma\big(\mathbf{w}^T\phi(\mathbf{x})\big)=0.5
\end{equation}
$

To plot this we need the inverse function of the logistic sigmoid which is 

$\begin{equation}
a = \ln\Big(\frac{\sigma}{1-\sigma}\Big)
\end{equation}
$

We plot the line $\mathbf{w}^T\phi(\mathbf{x})=0$. In this case  $\phi(x)=x$

Let's run the algorithm starting at $\mathbf{w} = [1, -1]$. We see that initially some examples are misclassified.


In [ ]:
# straight line
x1 = np.linspace(-2, 2, 100)
x2 = x1
for class_index in range(num_class):
    plt.plot(X[y==class_index, 0], 
             X[y==class_index, 1], 
             class_color[class_index] + 'o')
    plt.plot(x1, x2)
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel(r'$x_1$', fontsize=14)
    plt.xlabel(r'$x_2$', fontsize=14)

In [ ]:
weight_vector = np.array([1, -1])

for index in range(X.shape[0]):
    weight_vector = weight_vector - cross_entropy_gradient(X[index, :], 
                                                        weight_vector, 
                                                        y[index])


We can now plot the line corresponding to this learned weight vector and verify that it separates the classes correctly.

In [ ]:
# get the line
x2 = -x1 * (weight_vector[0] / weight_vector[1]) 

for class_index in range(num_class):
    plt.plot(X[y==class_index, 0], 
             X[y==class_index, 1], 
             class_color[class_index] + 'o')
    plt.plot(x1, x2)
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel(r'$x_1$', fontsize=14)
    plt.xlabel(r'$x_2$', fontsize=14)

Unlike the perceptron, logistic regression can be readily extended to the multiclass problem. See Bishop section 4.3.4.

## Reference
This notebook is based on the description of the perceptron algorithm in section 4.3.2 in Bishop's book - [Pattern Recognition and Machine Learning](https://www.springer.com/gp/book/9780387310732)